In [ ]:
!pip3 install git+https://github.com/lukasbahr/lena.git gwpy &> /dev/null
!pip3 install git+https://github.com/aliutkus/torchinterp1d.git gwpy &> /dev/null

In [ ]:
import sys ; sys.path.append('../')
from lena.observer.lueneberger import LuenebergerObserver
import lena.util.plot as plot
import torch
import math
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def getAutonomousSystem():
    # Define plant dynamics
    def f(x): return torch.cat((torch.reshape(torch.pow(x[1, :], 3), (1, -1)), torch.reshape(-x[0, :], (1, -1))), 0)
    def h(x): return torch.reshape(x[0, :], (1, -1))
    def g(x): return torch.zeros(x.shape[0], x.shape[1])
    def u(t): return 0

    # System dimension
    dim_x = 2
    dim_y = 1

    return f, h, g, u, dim_x, dim_y

In [ ]:
# Get system
f, h, g, u, dim_x, dim_y = getAutonomousSystem()

In [ ]:
# Setting up the Lueneberge observer

# Create Lueneberger observer
observer = LuenebergerObserver(dim_x, dim_y)

# Set observer functions
observer.f = f
observer.h = h
observer.g = g
observer.u = u

# Create eigenvalues for D from bessel filter
b, a = signal.bessel(3, 2*math.pi, 'low', analog=True, norm='phase')
eigen = np.roots(a)

# Set system dynamics of D and F
observer.D = observer.tensorDFromEigen(eigen)
observer.F = torch.Tensor([[1.0], [1.0], [1.0]])

In [ ]:
# Simulate sytem for a random initial condition

# Create initial value
y_0 = torch.tensor([[0.4,0.5,0,0,0]]).T

# Simulate forward in time starting from the last point from previous simulation
tsim = (0,20)
dt = 1e-2
tq_, data_ = observer.simulateSystem(y_0, tsim, dt)

In [ ]:
# Plot the simulated system
fig = plt.figure(dpi=100)

# Plot x
ax_x = fig.add_subplot(2, 1, 1)
ax_x.set_ylabel(r'$x$')
ax_x.set_xlabel('time' + r'$[s]$')
ax_x.plot(tq_, data_[:,:dim_x,0], color='blue', label='x')

# Plot z
ax_z = fig.add_subplot(2,1,2)
ax_z.set_ylabel(r'$z$')
ax_z.set_xlabel('time' + r'$[s]$')
ax_z.plot(tq_, data_[:,dim_x:,0])

plt.show()